In [1]:

import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import warnings
from typing import Dict, List, Tuple, Optional, Any
from sklearn.preprocessing import StandardScaler
from collections import deque
import random

warnings.filterwarnings('ignore')

In [2]:
# =============================================================================
# CONFIGURATION PARAMETERS
# =============================================================================

# Market parameters
TICK_VALUE = 10.0
MAX_CONTRACTS = 8
TRANSACTION_COST_PER_CONTRACT = 0.75
BID_ASK_SLIPPAGE = 0.5 # in ticks
SLIPPAGE_ENABLED = True

# Risk management
MAX_DAILY_LOSS = 500.0
STOP_LOSS_ENABLED = True
STOP_LOSS_PERCENTAGE = 0.3

# Data parameters
LOOKBACK_WINDOW = 20
MIN_LOOKBACK_FOR_TRAINING = 20
TRAIN_RATIO = 0.7
VAL_RATIO = 0.10
TEST_RATIO = 0.20

FEATURE_COLUMNS = [
   'PX_OPEN1', 'PX_HIGH1', 'PX_LOW1', 'PX_LAST1', 'PX_VOLUME1', 'OPEN_INT1',
   'PX_OPEN2', 'PX_HIGH2', 'PX_LOW2', 'PX_LAST2', 'PX_VOLUME2', 'OPEN_INT2',
   'VOL Change1', 'Vol Change %1', 'OI Change1', 'OI Change %1',
   'CALENDAR', 'Vol Ratio', 'Vol Ratio Change', 'OI Ratio', 'OI Ratio Change',
   'VOL Change2', 'Vol Change %2', 'OI Change2', 'OI Change %2'
]

TARGET_COLUMN = 'CALENDAR'
DATE_COLUMN = 'Dates'

# Goldman roll parameters
GOLDMAN_ROLL_START_DAY = 5
GOLDMAN_ROLL_END_DAY = 9
GOLDMAN_ROLL_WINDOW = 15

# LSTM parameters
LSTM_HIDDEN_SIZE = 32
LSTM_NUM_LAYERS = 2
LSTM_DROPOUT = 0.05
LSTM_BIDIRECTIONAL = False
LSTM_PROCESSING_DIM = 32
HIDDEN_DIMS = [32, 16]

# DDQN parameters
LEARNING_RATE = 0.0001
GAMMA = 0.99 #How much future rewards matter
TAU = 0.005  #How fast the "teacher network" updates
BATCH_SIZE = 16 #How many past trades to learn from at once


# Action space
POSITION_ACTIONS = [-0.5, -0.25, 0.0, 0.25, 0.5]
ACTION_SPACE_SIZE = len(POSITION_ACTIONS)

# Reward function
REWARD_PNL_WEIGHT = 0.7
REWARD_WINRATE_WEIGHT = 0.2
REWARD_RISK_WEIGHT = 0.1
DRAWDOWN_PENALTY_THRESHOLD = 0.1
POSITION_SIZE_PENALTY = 0.01

# Training parameters
EPISODES = 2000
MEMORY_SIZE = 500 #How many past trading experiences the agent remembers
UPDATE_FREQUENCY = 2 #How often the neural network actually trains
TARGET_UPDATE_FREQUENCY = 200 #How often the "teacher network" gets updated
EPSILON_START = 1.0
EPSILON_END = 0.2
EPSILON_DECAY = 0.995 #Reduce randomness by x% each episode
PATIENCE = 50 #Early stopping threshold
MIN_IMPROVEMENT = 0.001

# Device
DEVICE = torch.device("cuda")# if torch.cuda.is_available() else "cpu")

# Logging
LOG_FREQUENCY = 10
SAVE_FREQUENCY = 250
MODEL_SAVE_PATH = "models/"
LOG_SAVE_PATH = "logs/"
RESULTS_SAVE_PATH = "results/"

# Backtesting
INITIAL_CAPITAL = 10000.0
TRACK_METRICS = [
   'total_return', 'sharpe_ratio', 'max_drawdown', 'win_rate',
   'profit_factor', 'total_trades', 'avg_trade_duration'
]

# Quick test
QUICK_TEST_MODE = False
QUICK_TEST_YEARS = 3
QUICK_TEST_EPISODES = 100

# Plotting
PLOT_TRAINING_CURVES = True
PLOT_TRADING_RESULTS = True
SAVE_PLOTS = True

print(f"Configuration loaded successfully!")
print(f"Device: {DEVICE}")
print(f"Lookback window: {LOOKBACK_WINDOW} days")
print(f"Max contracts: {MAX_CONTRACTS}")
print(f"Tick value: ${TICK_VALUE}")
print(f"Quick test mode: {QUICK_TEST_MODE}")

In [3]:
# =============================================================================
# DATA HANDLER
# =============================================================================

class TradingDataHandler:
   def __init__(self, csv_path: str):
       self.csv_path = csv_path
       self.raw_data = None
       self.processed_data = None
       self.scaler = StandardScaler()
       self.feature_columns = FEATURE_COLUMNS
       self.date_column = DATE_COLUMN
       self.target_column = TARGET_COLUMN
       
   def load_data(self) -> pd.DataFrame:
       print(f"Loading data from {self.csv_path}...")
       
       column_names = [
           'PX_OPEN1', 'PX_HIGH1', 'PX_LOW1', 'PX_LAST1', 'PX_VOLUME1', 'OPEN_INT1',
           'PX_OPEN2', 'PX_HIGH2', 'PX_LOW2', 'PX_LAST2', 'PX_VOLUME2', 'OPEN_INT2',
           'Dates', 'VOL Change1', 'Vol Change %1', 'OI Change1', 'OI Change %1',
           'CALENDAR', 'Vol Ratio', 'Vol Ratio Change', 'OI Ratio', 'OI Ratio Change',
           'VOL Change2', 'Vol Change %2', 'OI Change2', 'OI Change %2'
           ]
       
       self.raw_data = pd.read_csv(self.csv_path, names=column_names, header=None, skiprows=3)
       
       self.raw_data['Dates'] = pd.to_datetime(self.raw_data['Dates'], dayfirst=True)
       self.raw_data = self.raw_data.sort_values('Dates').reset_index(drop=True)
       self.raw_data['date'] = self.raw_data['Dates']

        # to check if the data is loaded correctly
       print(f"Loaded {len(self.raw_data)} rows of data")
       print(f"Date range: {self.raw_data['Dates'].min()} to {self.raw_data['Dates'].max()}")
       print(f"CALENDAR spread stats:")
       print(f"  Min: {self.raw_data['CALENDAR'].min()}")
       print(f"  Max: {self.raw_data['CALENDAR'].max()}")
       print(f"  Mean: {self.raw_data['CALENDAR'].mean()}")
       print(f"  Std: {self.raw_data['CALENDAR'].std()}")
       
       return self.raw_data
   
   def calculate_business_day_features(self, df: pd.DataFrame) -> pd.DataFrame:
       df = df.copy()
       
       df['business_day_of_month'] = df['Dates'].apply(self._get_business_day_of_month)
       
       df['days_to_roll_start'] = df['business_day_of_month'].apply(
           lambda x: max(0, GOLDMAN_ROLL_START_DAY - x) if x < GOLDMAN_ROLL_START_DAY 
           else 0
       )
       
       df['days_since_roll_end'] = df['business_day_of_month'].apply(
           lambda x: max(0, x - GOLDMAN_ROLL_END_DAY) if x > GOLDMAN_ROLL_END_DAY 
           else 0
       )
       
       df['in_goldman_roll'] = (
           (df['business_day_of_month'] >= GOLDMAN_ROLL_START_DAY) & 
           (df['business_day_of_month'] <= GOLDMAN_ROLL_END_DAY)
       ).astype(int)
       
       df['in_extended_roll_window'] = (
           (df['business_day_of_month'] >= GOLDMAN_ROLL_START_DAY - 5) & 
           (df['business_day_of_month'] <= GOLDMAN_ROLL_END_DAY + 5)
       ).astype(int)
       
       return df
   
   def _get_business_day_of_month(self, date: datetime) -> int:
       first_day = date.replace(day=1)
       business_days = 0
       current_date = first_day
       
       while current_date <= date:
           if current_date.weekday() < 5:
               business_days += 1
           current_date += timedelta(days=1)
           
       return business_days
   
   def preprocess_data(self) -> pd.DataFrame:
       if self.raw_data is None:
           self.load_data()
           
       df = self.raw_data.copy()
       
       df = df.fillna(method='ffill').fillna(method='bfill')
       df = self.calculate_business_day_features(df)
       df = df.dropna()
       
       self.processed_data = df
       print(f"Preprocess complete final dataset: {len(df)} rows")
       
       return df
   
   def create_sequences(self, df: pd.DataFrame, lookback: int = None) -> Tuple[np.ndarray, np.ndarray, np.ndarray]:
       if lookback is None:
           lookback = LOOKBACK_WINDOW
       
       feature_cols = [col for col in df.columns if col not in ['date', 'Dates']]
       feature_cols = [col for col in feature_cols if not col.startswith('business_day')]
       feature_cols = [col for col in feature_cols if not col.startswith('days_')]
       feature_cols = [col for col in feature_cols if not col.startswith('in_goldman')]
       feature_cols = [col for col in feature_cols if not col.startswith('in_extended')]
   
    
   
       feature_data = self.scaler.fit_transform(df[feature_cols])
   
       sequences = []
       targets = []
       dates = []
   
       for i in range(lookback, len(df)):
           sequence = feature_data[i-lookback:i].T
           sequences.append(sequence)
           targets.append(df.iloc[i][self.target_column])
           dates.append(df.iloc[i]['Dates'])
   
       return np.array(sequences), np.array(targets), np.array(dates)
   
   def split_data(self, sequences: np.ndarray, targets: np.ndarray, dates: np.ndarray, 
                  quick_test: bool = False) -> Dict:
       
       if quick_test:
           n_years = QUICK_TEST_YEARS
           cutoff_date = dates[-1] - pd.DateOffset(years=n_years)
           mask = pd.to_datetime(dates) >= cutoff_date
           
           sequences = sequences[mask]
           targets = targets[mask]
           dates = dates[mask]
           
       n_samples = len(sequences)
       
       train_end = int(n_samples * TRAIN_RATIO)
       val_end = int(n_samples * (TRAIN_RATIO + VAL_RATIO))
       
       data_splits = {
           'train': {
               'sequences': sequences[:train_end],
               'targets': targets[:train_end],
               'dates': dates[:train_end]
           },
           'val': {
               'sequences': sequences[train_end:val_end],
               'targets': targets[train_end:val_end],
               'dates': dates[train_end:val_end]
           },
           'test': {
               'sequences': sequences[val_end:],
               'targets': targets[val_end:],
               'dates': dates[val_end:]
           }
       }
       
       # to check if the data splits are correct
       print(f"Data splits:")
       print(f"  Train: {len(data_splits['train']['sequences'])} samples")
       print(f"  Validation: {len(data_splits['val']['sequences'])} samples") 
       print(f"  Test: {len(data_splits['test']['sequences'])} samples")
       
       return data_splits
   
   def get_feature_info(self) -> Dict:
       if self.processed_data is None:
           self.preprocess_data()
           
       feature_cols = [col for col in self.processed_data.columns if col not in ['date']]
       
       return {
           'n_features': len(feature_cols),
           'feature_names': feature_cols,
           'n_samples': len(self.processed_data),
           'date_range': (self.processed_data['date'].min(), self.processed_data['date'].max())
       }


In [ ]:
# =============================================================================
# NEURAL NETWORKS
# =============================================================================

class LSTMFeatureExtractor(nn.Module):
   def __init__(self, n_features: int, sequence_length: int):
       super(LSTMFeatureExtractor, self).__init__()
       
       self.n_features = n_features
       self.sequence_length = sequence_length
       
       self.hidden_size = LSTM_HIDDEN_SIZE
       self.num_layers = LSTM_NUM_LAYERS
       self.dropout = LSTM_DROPOUT
       
       self.lstm = nn.LSTM(
           input_size=n_features,
           hidden_size=self.hidden_size,
           num_layers=self.num_layers,
           dropout=self.dropout if self.num_layers > 1 else 0,
           batch_first=True,
           bidirectional=LSTM_BIDIRECTIONAL
       )
       
       self.lstm_output_size = self.hidden_size * (2 if LSTM_BIDIRECTIONAL else 1)
       
       self.feature_processor = nn.Sequential(
           nn.Linear(self.lstm_output_size, LSTM_PROCESSING_DIM),
           nn.ReLU(),
           nn.Dropout(LSTM_DROPOUT),
           nn.Linear(LSTM_PROCESSING_DIM, LSTM_PROCESSING_DIM // 2),
           nn.ReLU(),
           nn.Dropout(LSTM_DROPOUT)
       )
       
       self.output_dim = LSTM_PROCESSING_DIM // 2
       
   def forward(self, x):
       batch_size = x.size(0)
       h0 = torch.zeros(
           self.num_layers * (2 if LSTM_BIDIRECTIONAL else 1),
           batch_size,
           self.hidden_size,
           device=x.device
       )
       c0 = torch.zeros(
           self.num_layers * (2 if LSTM_BIDIRECTIONAL else 1),
           batch_size,
           self.hidden_size,
           device=x.device
       )
       
       lstm_out, (hn, cn) = self.lstm(x, (h0, c0))
       final_output = lstm_out[:, -1, :]
       features = self.feature_processor(final_output)
       
       return features

class DQNNetwork(nn.Module):
   def __init__(self, market_feature_dim: int, trading_state_dim: int, 
                action_space_size: int, sequence_length: int):
       super(DQNNetwork, self).__init__()
       
       self.market_feature_dim = market_feature_dim
       self.trading_state_dim = trading_state_dim
       self.action_space_size = action_space_size
       
       self.lstm = LSTMFeatureExtractor(market_feature_dim, sequence_length)
       
       total_input_dim = self.lstm.output_dim + trading_state_dim
       
       self.fc_layers = nn.ModuleList()
       
       self.fc_layers.append(nn.Linear(total_input_dim, HIDDEN_DIMS[0]))
       
       for i in range(1, len(HIDDEN_DIMS)):
           self.fc_layers.append(
               nn.Linear(HIDDEN_DIMS[i-1], HIDDEN_DIMS[i])
           )
       
       self.q_values = nn.Linear(HIDDEN_DIMS[-1], action_space_size)
       
       self.dropout = nn.Dropout(LSTM_DROPOUT)
       
       self._initialize_weights()
   
   def _initialize_weights(self):
       for module in self.modules():
           if isinstance(module, nn.Linear):
               nn.init.xavier_uniform_(module.weight)
               if module.bias is not None:
                   nn.init.constant_(module.bias, 0)
           elif isinstance(module, nn.LSTM):
               for name, param in module.named_parameters():
                   if 'weight_ih' in name:
                       nn.init.xavier_uniform_(param.data)
                   elif 'weight_hh' in name:
                       nn.init.orthogonal_(param.data)
                   elif 'bias' in name:
                       param.data.fill_(0)
   
   def forward(self, market_features, trading_state):
       lstm_features = self.lstm(market_features)
       
       combined_features = torch.cat([lstm_features, trading_state], dim=1)
       
       x = combined_features
       for fc_layer in self.fc_layers:
           x = fc_layer(x)
           x = F.relu(x)
           x = self.dropout(x)
       
       q_values = self.q_values(x)
       
       return q_values

class DoubleDQN(nn.Module):
   def __init__(self, market_feature_dim: int, trading_state_dim: int,
                action_space_size: int, sequence_length: int):
       super(DoubleDQN, self).__init__()
       
       self.online_net = DQNNetwork(
           market_feature_dim, trading_state_dim, action_space_size, sequence_length
       )
       
       self.target_net = DQNNetwork(
           market_feature_dim, trading_state_dim, action_space_size, sequence_length
       )
       
       self.update_target_network()
       
       for param in self.target_net.parameters():
           param.requires_grad = False
   
   def forward(self, market_features, trading_state, use_target=False):
       if use_target:
           return self.target_net(market_features, trading_state)
       else:
           return self.online_net(market_features, trading_state)
   
   def update_target_network(self, tau: float = None):
       if tau is None:
           self.target_net.load_state_dict(self.online_net.state_dict())
       else:
           for target_param, online_param in zip(
               self.target_net.parameters(), self.online_net.parameters()
           ):
               target_param.data.copy_(
                   tau * online_param.data + (1.0 - tau) * target_param.data
               )
   
   def get_action(self, market_features, trading_state, epsilon=0.0):
       if np.random.random() < epsilon:
           return np.random.randint(0, self.online_net.action_space_size)
       else:
           with torch.no_grad():
               q_values = self.online_net(market_features, trading_state)
               return q_values.argmax().item()

class ReplayBuffer:
   def __init__(self, capacity: int):
       self.capacity = capacity
       self.buffer = []
       self.position = 0
   
   def push(self, market_features, trading_state, action, reward, 
            next_market_features, next_trading_state, done):
       if len(self.buffer) < self.capacity:
           self.buffer.append(None)
       
       self.buffer[self.position] = (
           market_features, trading_state, action, reward,
           next_market_features, next_trading_state, done
       )
       self.position = (self.position + 1) % self.capacity
   
   def sample(self, batch_size: int):
       batch = np.random.choice(len(self.buffer), batch_size, replace=False)
       
       market_features = []
       trading_states = []
       actions = []
       rewards = []
       next_market_features = []
       next_trading_states = []
       dones = []
       
       for idx in batch:
           mf, ts, a, r, nmf, nts, d = self.buffer[idx]
           market_features.append(mf)
           trading_states.append(ts)
           actions.append(a)
           rewards.append(r)
           next_market_features.append(nmf)
           next_trading_states.append(nts)
           dones.append(d)
       
       return (
           torch.FloatTensor(market_features),
           torch.FloatTensor(trading_states),
           torch.LongTensor(actions),
           torch.FloatTensor(rewards),
           torch.FloatTensor(next_market_features),
           torch.FloatTensor(next_trading_states),
           torch.BoolTensor(dones)
       )
   
   def __len__(self):
       return len(self.buffer)

def save_model(model: DoubleDQN, filepath: str, optimizer_state: dict = None, 
              metadata: dict = None):
   checkpoint = {
       'online_net_state_dict': model.online_net.state_dict(),
       'target_net_state_dict': model.target_net.state_dict(),
   }
   
   if optimizer_state:
       checkpoint['optimizer_state_dict'] = optimizer_state
   
   if metadata:
       checkpoint['metadata'] = metadata
   
   torch.save(checkpoint, filepath)
   print(f"Model saved to {filepath}")

def load_model(model: DoubleDQN, filepath: str, load_optimizer: bool = False):
   checkpoint = torch.load(filepath, map_location=DEVICE)
   
   model.online_net.load_state_dict(checkpoint['online_net_state_dict'])
   model.target_net.load_state_dict(checkpoint['target_net_state_dict'])
   
   optimizer_state = None
   if load_optimizer and 'optimizer_state_dict' in checkpoint:
       optimizer_state = checkpoint['optimizer_state_dict']
   
   metadata = checkpoint.get('metadata', {})
   
   print(f"Model loaded from {filepath}")
   return optimizer_state, metadata

In [5]:
# =============================================================================
# TRADING ENVIRONMENT
# =============================================================================

class TradingEnvironment:
    def __init__(self, data_sequences: np.ndarray, data_targets: np.ndarray, 
                 data_dates: np.ndarray, initial_capital: float = None):
        self.data_sequences = data_sequences
        self.data_targets = data_targets
        self.data_dates = data_dates
        self.n_samples = len(data_sequences)
        
        self.initial_capital = initial_capital or INITIAL_CAPITAL
        self.tick_value = TICK_VALUE
        self.max_contracts = MAX_CONTRACTS
        self.transaction_cost = TRANSACTION_COST_PER_CONTRACT
        self.slippage = BID_ASK_SLIPPAGE if SLIPPAGE_ENABLED else 0.0
        
        self.max_daily_loss = MAX_DAILY_LOSS
        self.stop_loss_enabled = STOP_LOSS_ENABLED
        self.stop_loss_pct = STOP_LOSS_PERCENTAGE
        
        self.reset()

    def debug_pnl_calculation(self):

        trade_pnl_sum = sum(trade['pnl'] for trade in self.trade_history)
        expected_equity = self.initial_capital + trade_pnl_sum
        actual_equity = self.cash + self.unrealized_pnl
    
        print(f"Trade P&L Sum: ${trade_pnl_sum:,.2f}")
        print(f"Expected Equity: ${expected_equity:,.2f}")
        print(f"Actual Equity: ${actual_equity:,.2f}")
        print(f"Missing: ${actual_equity - expected_equity:,.2f}")
        print(f"Transaction Costs: ${getattr(self, 'total_transaction_costs', 0):,.2f}")
    
    def _get_business_day_of_month(self, date) -> int:
        if isinstance(date, str):
             date = pd.to_datetime(date)
 
        first_day = date.replace(day=1)
        business_days = 0
        current_date = first_day
 
        while current_date <= date:
             if current_date.weekday() < 5:  # Monday = 0, Friday = 4
                 business_days += 1
             current_date += pd.Timedelta(days=1)
         
        return business_days
 
        
    def reset(self) -> np.ndarray:
        self.current_step = 0
        self.position = 0
        self.cash = self.initial_capital
        self.total_pnl = 0.0
        self.unrealized_pnl = 0.0
        self.daily_pnl = 0.0
        
        # trade tracking
        self.winning_trades = 0
        self.losing_trades = 0
        self.total_trades = 0
        self.all_trade_pnls = []  
        
        # Position tracking
        self.position_entry_price = None
        self.position_entry_step = None
        self.last_position = 0
        
        self.trade_history = []
        self.equity_curve = [self.initial_capital]
        self.daily_returns = []
        self.max_drawdown = 0.0
        self.peak_equity = self.initial_capital
        
        self.daily_loss_tracker = 0.0
        self.stop_loss_triggered = False
        
        
        return self._get_state()
    
    def _get_state(self) -> np.ndarray:
        if self.current_step >= self.n_samples:
            market_features = np.zeros((self.data_sequences.shape[1], self.data_sequences.shape[2]))
        else:
            market_features = self.data_sequences[self.current_step]
        
        market_state = market_features.flatten()
        
        trading_state = np.array([
            self.position / self.max_contracts,
            self.unrealized_pnl / self.initial_capital,
            self.total_pnl / self.initial_capital,
            self.daily_pnl / self.initial_capital,
            (self.cash + self.unrealized_pnl) / self.initial_capital,
            self.max_drawdown,
            float(self.stop_loss_triggered),
            self.current_step / self.n_samples,
        ])
        
        return np.concatenate([market_state, trading_state])
    


    def step(self, action: int) -> Tuple[np.ndarray, float, bool, Dict]:
    
        if self.current_step >= self.n_samples - 1:
            if self.position != 0:
                self.close_final_position()
            return self._get_state(), 0.0, True, {'reason': 'end_of_data'}
    
        current_price = self.data_targets[self.current_step]
        next_price = self.data_targets[self.current_step + 1]

        oi_ratio_feature_index = 15
        self.current_oi_ratio = self.data_sequences[self.current_step][oi_ratio_feature_index][-1]

        position_change_pct = POSITION_ACTIONS[action]
        position_change = int(position_change_pct * self.max_contracts)

        new_position = np.clip(
            self.position + position_change, 
            -self.max_contracts, 
            self.max_contracts
        )

        # Position hold timer logic
        current_date = self.data_dates[self.current_step]
        current_bday = self._get_business_day_of_month(current_date)

        if not hasattr(self, 'position_hold_timer'):
            self.position_hold_timer = 0
            self.forced_closes = 0

        if self.position != 0:
            self.position_hold_timer += 1
            if self.position_hold_timer >= 10:
                new_position = 0
                self.forced_closes += 1
                self.position_hold_timer = 0
        else:
            self.position_hold_timer = 0
    
        actual_position_change = new_position - self.position
    
        # Calculate transaction costs
        transaction_cost = abs(actual_position_change) * self.transaction_cost
        slippage_cost = abs(actual_position_change) * self.slippage * self.tick_value
        total_cost = transaction_cost + slippage_cost

        #if total_cost > 0:
         #   print(f"Action {action}: {abs(actual_position_change)} contracts × cost = ${total_cost:.2f}")
    
        # Track total transaction costs
        if not hasattr(self, 'total_transaction_costs'):
            self.total_transaction_costs = 0
        self.total_transaction_costs += total_cost
    
        old_position = self.position
    
        if self.position != 0 and self.position_entry_price is not None:
            # Calculate current unrealized P&L
            price_change_since_entry = current_price - self.position_entry_price
            current_unrealized = self.position * price_change_since_entry * self.tick_value
        
            # If position is changing, to account for the P&L
            if actual_position_change != 0:
                pass  # Let handle_position_change deal with this
    
        # Record trade if position changes
        if actual_position_change != 0:
            self.handle_position_change(old_position, new_position, current_price)
    
        # Update position and cash
        self.position = new_position
        self.cash -= total_cost
    
        self.current_step += 1
    
        if self.position != 0 and self.position_entry_price is not None:
            # Calculate unrealized P&L based on current position
            price_change_since_entry = next_price - self.position_entry_price
            self.unrealized_pnl = self.position * price_change_since_entry * self.tick_value
        else:
            # No position = no unrealized P&L
            self.unrealized_pnl = 0.0
    
        # Daily P&L is just the change in total equity
        current_equity = self.cash + self.unrealized_pnl
        previous_equity = self.equity_curve[-1] if self.equity_curve else self.initial_capital
        self.daily_pnl = current_equity - previous_equity
    
        self.equity_curve.append(current_equity)
    
        # Update peak and drawdown
        if current_equity > self.peak_equity:
            self.peak_equity = current_equity
    
        current_drawdown = (self.peak_equity - current_equity) / self.peak_equity
        self.max_drawdown = max(self.max_drawdown, current_drawdown)
    
        done = False
        info = {'transaction_cost': total_cost}
    
        # Risk checks
        if self.daily_pnl < -self.max_daily_loss:
            done = True
            info['reason'] = 'daily_loss_limit'
            self.stop_loss_triggered = True
    
        if self.stop_loss_enabled and current_drawdown > self.stop_loss_pct:
            done = True
            info['reason'] = 'stop_loss'
            self.stop_loss_triggered = True
    
        reward = self.calculate_reward()
    
        # Reset daily P&L periodically
        if self.current_step % 1 == 0:
            self.daily_pnl = 0.0
    
        next_state = self._get_state()
    
        # Enhanced info
        current_win_rate = self.winning_trades / max(1, self.total_trades)
    
        info.update({
            'position': self.position,
            'cash': self.cash,
            'unrealized_pnl': self.unrealized_pnl,
            'total_pnl': self.total_pnl,
            'equity': current_equity,
            'drawdown': current_drawdown,
            'transaction_cost': total_cost,
            'current_price': next_price,
            'total_trades': self.total_trades,
            'winning_trades': self.winning_trades,
            'losing_trades': self.losing_trades,
            'win_rate': current_win_rate,
            'total_transaction_costs': getattr(self, 'total_transaction_costs', 0)
        })
    
        return next_state, reward, done, info

    def handle_position_change(self, old_position: int, new_position: int, current_price: float):
        
        self.last_position = old_position  # Make sure this line exists


        # Case 1: Going from zero to non-zero (opening position)
        if old_position == 0 and new_position != 0:
            self.position_entry_price = current_price
            self.position_entry_step = self.current_step
            #print(f"Step {self.current_step}: Opening position {new_position} at ${current_price:.2f}")
            
        # Case 2: Going from non-zero to zero (closing position)
        elif old_position != 0 and new_position == 0:
            if self.position_entry_price is not None:
                trade_pnl = (current_price - self.position_entry_price) * old_position * self.tick_value
                self.record_simple_trade(trade_pnl, current_price)
                #print(f"Step {self.current_step}: Closing position {old_position} at ${current_price:.2f}, P&L: ${trade_pnl:.2f}")
            
        # Case 3: Flipping from long to short or vice versa
        elif (old_position > 0 and new_position < 0) or (old_position < 0 and new_position > 0):
            if self.position_entry_price is not None:
                # Close the old position
                trade_pnl = (current_price - self.position_entry_price) * old_position * self.tick_value
                self.record_simple_trade(trade_pnl, current_price)
                #print(f"Step {self.current_step}: Flipping position {old_position} to {new_position}, P&L: ${trade_pnl:.2f}")
            
            # Open new position in opposite direction
            self.position_entry_price = current_price
            self.position_entry_step = self.current_step
            
        # Case 4: Just changing size in same direction (partial close/add)
        elif abs(new_position) < abs(old_position):
            # Reducing position size - record partial trade
            if self.position_entry_price is not None:
                size_reduction = abs(old_position) - abs(new_position)
                trade_pnl = (current_price - self.position_entry_price) * size_reduction * (1 if old_position > 0 else -1) * self.tick_value
                self.record_simple_trade(trade_pnl, current_price)
                #print(f"Step {self.current_step}: Reducing position by {size_reduction}, P&L: ${trade_pnl:.2f}")
    
    def record_simple_trade(self, trade_pnl: float, exit_price: float):
    
            self.total_trades += 1
            self.all_trade_pnls.append(trade_pnl)
    
            # Simple classification
            if trade_pnl > 0.01:
                self.winning_trades += 1
                result = "WIN"
            elif trade_pnl < -0.01:
                self.losing_trades += 1
                result = "LOSS"
            else:
                result = "BREAK-EVEN"
    
            # Update total realized P&L
            self.total_pnl += trade_pnl
    
            
            self.unrealized_pnl = 0.0
    
            # Store trade info
            trade_info = {
            'entry_price': self.position_entry_price,
            'exit_price': exit_price,
            'entry_step': self.position_entry_step,
            'exit_step': self.current_step,
            'pnl': trade_pnl,
            'position_size': getattr(self, 'last_position', 0),
            'result': result,
            'duration': self.current_step - (self.position_entry_step or self.current_step)
            }
            self.trade_history.append(trade_info)

            
    def close_final_position(self):
        #Close any remaining position at episode end
        if self.position != 0 and self.position_entry_price is not None:
            final_price = self.data_targets[self.current_step]
            trade_pnl = (final_price - self.position_entry_price) * self.position * self.tick_value
            self.record_simple_trade(trade_pnl, final_price)
            #print(f"Final position close: P&L ${trade_pnl:.2f}")
            self.position = 0
    
    
    #============== REWARD FORMULA ==============
    
    
    def calculate_reward(self) -> float:
        current_equity = self.cash + self.unrealized_pnl
    
        # P&L component (normalised by initial capital)
        pnl_reward = self.daily_pnl / self.initial_capital

        # OI Ratio constraint penalty
        oi_penalty = 0.0
        if hasattr(self, 'current_oi_ratio') and self.current_oi_ratio >= 1.5:
            if self.position != 0:
                oi_penalty = 5.0  # Strong penalty for trading when OI ratio >= 1.5
            else:
                oi_penalty = -1  # Small bonus for staying flat when OI ratio >= 1.5
        if hasattr(self, 'current_oi_ratio') and self.current_oi_ratio < 0.9:
            if self.position != 0:
                oi_penalty = -2.0  # Bonus for trading when OI ratio < 0.8 (good time)

        # Risk penalty component
        risk_penalty = 0.0
        if self.max_drawdown > DRAWDOWN_PENALTY_THRESHOLD:
            risk_penalty += (self.max_drawdown - DRAWDOWN_PENALTY_THRESHOLD) * 10

        # Combine components (removed win rate)
        reward = (
            REWARD_PNL_WEIGHT * pnl_reward -
            REWARD_RISK_WEIGHT * risk_penalty -
            oi_penalty
    )   

        return reward
    
#=================================================================

    def get_performance_metrics(self) -> Dict:
        if len(self.equity_curve) < 2:
            return {}
        
        # Close any remaining position for final calculations
        self.close_final_position()
        
        returns = np.diff(self.equity_curve) / self.equity_curve[:-1]
        total_return = (self.equity_curve[-1] - self.initial_capital) / self.initial_capital
        
        if len(returns) > 1 and np.std(returns) > 0:
            sharpe_ratio = np.mean(returns) / np.std(returns) * np.sqrt(252)
        else:
            sharpe_ratio = 0.0
        
        #  win rate calculation
        win_rate = self.winning_trades / max(1, self.total_trades)
        
        # Calculate profit factor
        winning_pnl = sum([pnl for pnl in self.all_trade_pnls if pnl > 0])
        losing_pnl = abs(sum([pnl for pnl in self.all_trade_pnls if pnl < 0]))
        
        if losing_pnl > 0:
            profit_factor = winning_pnl / losing_pnl
        elif winning_pnl > 0:
            profit_factor = float('inf')
        else:
            profit_factor = 0.0
        
        if self.trade_history:
            avg_trade_duration = np.mean([trade['duration'] for trade in self.trade_history])
        else:
            avg_trade_duration = 0.0
        
        # Debug output
        #print(f"\n=== FINAL PERFORMANCE METRICS ===")
        #print(f"Total trades recorded: {self.total_trades}")
        #print(f"Winning trades: {self.winning_trades}")
        #print(f"Losing trades: {self.losing_trades}")
        #print(f"Win rate calculation: {self.winning_trades} / {self.total_trades} = {win_rate:.1%}")
        #print(f"All trade P&Ls: {self.all_trade_pnls}")
        #print(f"Trade history entries: {len(self.trade_history)}")
        
        return {
            'total_return': total_return,
            'sharpe_ratio': sharpe_ratio,
            'max_drawdown': self.max_drawdown,
            'win_rate': win_rate,
            'profit_factor': profit_factor,
            'total_trades': self.total_trades,
            'winning_trades': self.winning_trades,
            'losing_trades': self.losing_trades,
            'avg_trade_duration': avg_trade_duration,
            'final_equity': self.equity_curve[-1],
            'total_pnl': self.total_pnl,
            'unrealized_pnl': self.unrealized_pnl
        }
    
    def get_action_space_size(self) -> int:
        return len(POSITION_ACTIONS)
    
    def get_state_size(self) -> int:
        if self.current_step < self.n_samples:
            market_features = self.data_sequences[self.current_step].flatten()
        else:
            market_features = np.zeros((self.data_sequences.shape[1] * self.data_sequences.shape[2]))
        
        trading_features = 8
        return len(market_features) + trading_features
    
    def render(self, mode='human'):
        if mode == 'human':
            print(f"Step: {self.current_step}")
            print(f"Position: {self.position} contracts")
            print(f"Cash: ${self.cash:,.2f}")
            print(f"Unrealized P&L: ${self.unrealized_pnl:,.2f}")
            print(f"Total P&L: ${self.total_pnl:,.2f}")
            print(f"Equity: ${self.cash + self.unrealized_pnl:,.2f}")
            print(f"Drawdown: {self.max_drawdown:.2%}")
            print(f"Trades: {self.total_trades} (Wins: {self.winning_trades}, Losses: {self.losing_trades})")
            print(f"Win Rate: {self.winning_trades/max(1,self.total_trades):.1%}")
            print("-" * 50)
    
    def _calculate_episode_end_bonus(self) -> float:
    
        # Win rate bonus
        win_rate_bonus = 0.0
        if self.total_trades >= 5:  # Only if we have sufficient trades
            win_rate = self.winning_trades / self.total_trades
            # Scale the bonus: +0.5 for 100% win rate, 0 for 50% win rate, -0.5 for 0% win rate
            win_rate_bonus = REWARD_WINRATE_WEIGHT * (win_rate - 0.5) * 2
    
        # Optional: Add other episode-end performance bonuses
        total_return_bonus = 0.0
        if len(self.equity_curve) > 1:
            total_return = (self.equity_curve[-1] - self.initial_capital) / self.initial_capital
            # Small bonus for positive returns
            total_return_bonus = max(0, total_return * 0.1)
    
        # Combine episode-end bonuses
        total_bonus = win_rate_bonus + total_return_bonus
    
        return total_bonus


class TradingEnvironmentWrapper:
    def __init__(self, data_splits: Dict):
        self.data_splits = data_splits
        
    def create_env(self, split: str = 'train') -> TradingEnvironment:
        if split not in self.data_splits:
            raise ValueError(f"Split '{split}' not found. Available: {list(self.data_splits.keys())}")
        
        data = self.data_splits[split]
        return TradingEnvironment(
            data_sequences=data['sequences'],
            data_targets=data['targets'],
            data_dates=data['dates']
        )
    
    def get_env_info(self) -> Dict:
        info = {}
        for split in self.data_splits:
            env = self.create_env(split)
            info[split] = {
                'n_samples': env.n_samples,
                'state_size': env.get_state_size(),
                'action_space_size': env.get_action_space_size(),
                'date_range': (env.data_dates[0], env.data_dates[-1])
            }
        return info

In [6]:
# =============================================================================
# DDQN AGENT
# =============================================================================

class DDQNTradingAgent:
   def __init__(self, market_feature_dim: int, trading_state_dim: int, 
                action_space_size: int, lookback_window: int):
       self.market_feature_dim = market_feature_dim
       self.trading_state_dim = trading_state_dim
       self.action_space_size = action_space_size
       self.lookback_window = lookback_window
       
       self.ddqn = DoubleDQN(
           market_feature_dim, trading_state_dim, action_space_size, lookback_window
       ).to(DEVICE)
       
       self.optimizer = optim.Adam(
           self.ddqn.online_net.parameters(), 
           lr=LEARNING_RATE
       )
       
       self.criterion = nn.MSELoss()
       
       self.replay_buffer = ReplayBuffer(MEMORY_SIZE)
       
       self.epsilon = EPSILON_START
       self.epsilon_min = EPSILON_END
       self.epsilon_decay = EPSILON_DECAY
       self.gamma = GAMMA
       self.tau = TAU
       self.batch_size = BATCH_SIZE
       self.update_frequency = UPDATE_FREQUENCY
       self.target_update_frequency = TARGET_UPDATE_FREQUENCY
       
       self.training_step = 0
       self.episode_rewards = []
       self.episode_losses = []
       self.episode_epsilons = []
       self.training_metrics = {
           'rewards': [],
           'losses': [],
           'epsilons': [],
           'q_values': [],
           'actions_taken': []
       }
       
       print(f"DDQN Agent initialised {self._count_parameters()} parameters")
       print(f"Device: {DEVICE}")
   
   def _count_parameters(self) -> int:
       return sum(p.numel() for p in self.ddqn.online_net.parameters() if p.requires_grad)
   
   def _state_to_tensors(self, state: np.ndarray) -> Tuple[torch.Tensor, torch.Tensor]:
       market_size = self.market_feature_dim * self.lookback_window
       market_features = state[:market_size].reshape(self.market_feature_dim, self.lookback_window)
       trading_state = state[market_size:]
       
       market_tensor = torch.FloatTensor(market_features).permute(1, 0).unsqueeze(0).to(DEVICE)
       trading_tensor = torch.FloatTensor(trading_state).unsqueeze(0).to(DEVICE)
       
       return market_tensor, trading_tensor
   
   def get_action(self, state: np.ndarray, training: bool = True) -> int:
       if training and random.random() < self.epsilon:
           action = random.randint(0, self.action_space_size - 1)
           self.training_metrics['actions_taken'].append(('random', action))
       else:
           market_features, trading_state = self._state_to_tensors(state)
           
           with torch.no_grad():
               q_values = self.ddqn(market_features, trading_state, use_target=False)
               action = q_values.argmax().item()
               
               if training:
                   self.training_metrics['q_values'].append(q_values.cpu().numpy()[0])
                   self.training_metrics['actions_taken'].append(('greedy', action))
       
       return action
   
   def store_transition(self, state: np.ndarray, action: int, reward: float,
                       next_state: np.ndarray, done: bool):
       market_features, trading_state = self._state_to_tensors(state)
       next_market_features, next_trading_state = self._state_to_tensors(next_state)
       
       self.replay_buffer.push(
           market_features.squeeze(0).cpu().numpy(),
           trading_state.squeeze(0).cpu().numpy(),
           action,
           reward,
           next_market_features.squeeze(0).cpu().numpy(),
           next_trading_state.squeeze(0).cpu().numpy(),
           done
       )
   
   def train_step(self) -> float:
       if len(self.replay_buffer) < self.batch_size:
           return 0.0
       
       batch = self.replay_buffer.sample(self.batch_size)
       market_features, trading_states, actions, rewards, next_market_features, next_trading_states, dones = batch
       
       market_features = market_features.to(DEVICE)
       trading_states = trading_states.to(DEVICE)
       actions = actions.to(DEVICE)
       rewards = rewards.to(DEVICE)
       next_market_features = next_market_features.to(DEVICE)
       next_trading_states = next_trading_states.to(DEVICE)
       dones = dones.to(DEVICE)
       
       current_q_values = self.ddqn(market_features, trading_states, use_target=False)
       current_q_values = current_q_values.gather(1, actions.unsqueeze(1)).squeeze(1)
       
       with torch.no_grad():
           next_q_values_online = self.ddqn(next_market_features, next_trading_states, use_target=False)
           next_actions = next_q_values_online.argmax(1)
           
           next_q_values_target = self.ddqn(next_market_features, next_trading_states, use_target=True)
           next_q_values = next_q_values_target.gather(1, next_actions.unsqueeze(1)).squeeze(1)
           
           target_q_values = rewards + (self.gamma * next_q_values * ~dones)
       
       loss = self.criterion(current_q_values, target_q_values)
       
       self.optimizer.zero_grad()
       loss.backward()
       
       torch.nn.utils.clip_grad_norm_(self.ddqn.online_net.parameters(), 1.0)
       
       self.optimizer.step()
       
       if self.training_step % self.target_update_frequency == 0:
           self.ddqn.update_target_network(self.tau)
       
       if self.epsilon > self.epsilon_min:
           self.epsilon *= self.epsilon_decay
       
       self.training_step += 1
       
       return loss.item()
   
   def train_episode(self, env, episode: int) -> Dict:
       state = env.reset()
       total_reward = 0.0
       total_loss = 0.0
       steps = 0
       episode_actions = []
       
       while True:
           action = self.get_action(state, training=True)
           episode_actions.append(action)
           
           next_state, reward, done, info = env.step(action)
           
           self.store_transition(state, action, reward, next_state, done)
           
           if len(self.replay_buffer) >= self.batch_size and steps % self.update_frequency == 0:
               loss = self.train_step()
               total_loss += loss
           
           total_reward += reward
           steps += 1
           state = next_state
           
           if done:
               break
       
       episode_metrics = {
           'episode': episode,
           'total_reward': total_reward,
           'average_loss': total_loss / max(1, steps // self.update_frequency),
           'steps': steps,
           'epsilon': self.epsilon,
           'actions': episode_actions,
           'final_performance': env.get_performance_metrics()
       }
       
       self.episode_rewards.append(total_reward)
       self.episode_losses.append(total_loss / max(1, steps // self.update_frequency))
       self.episode_epsilons.append(self.epsilon)
       
       return episode_metrics
   
   def evaluate_episode(self, env) -> Dict:
       state = env.reset()
       total_reward = 0.0
       steps = 0
       episode_actions = []
       
       while True:
           action = self.get_action(state, training=False)
           episode_actions.append(action)
           
           next_state, reward, done, info = env.step(action)
           
           total_reward += reward
           steps += 1
           state = next_state
           
           if done:
               break
       
       performance_metrics = env.get_performance_metrics()
       
       eval_metrics = {
           'total_reward': total_reward,
           'steps': steps,
           'actions': episode_actions,
           'performance': performance_metrics
       }
       
       return eval_metrics
   
   def train(self, train_env, val_env, num_episodes: int = None) -> Dict:
       if num_episodes is None:
           num_episodes = EPISODES
       
       training_history = {
           'train_rewards': [],
           'val_rewards': [],
           'train_performance': [],
           'val_performance': [],
           'losses': [],
           'epsilons': []
       }
       
       best_val_reward = float('-inf')
       patience_counter = 0
       
       print(f"training for {num_episodes} episodes...")
       print(f"replay buffer: {MEMORY_SIZE}")
       print(f"batch size: {self.batch_size}")
       print(f"update frequency: {self.update_frequency}")
       
       for episode in range(num_episodes):
           train_metrics = self.train_episode(train_env, episode)
           
           if episode % LOG_FREQUENCY == 0:
               val_metrics = self.evaluate_episode(val_env)
               
               training_history['train_rewards'].append(train_metrics['total_reward'])
               training_history['val_rewards'].append(val_metrics['total_reward'])
               training_history['train_performance'].append(train_metrics['final_performance'])
               training_history['val_performance'].append(val_metrics['performance'])
               training_history['losses'].append(train_metrics['average_loss'])
               training_history['epsilons'].append(train_metrics['epsilon'])
               
               print(f"Episode {episode}/{num_episodes}")
               print(f"  Train Reward: {train_metrics['total_reward']:.4f}")
               print(f"  Val Reward: {val_metrics['total_reward']:.4f}")
               print(f"  Loss: {train_metrics['average_loss']:.6f}")
               print(f"  Epsilon: {train_metrics['epsilon']:.4f}")
               print(f"  Val Performance: {val_metrics['performance'].get('total_return', 0):.2%}")
               print("-" * 50)
               
               if val_metrics['total_reward'] > best_val_reward + MIN_IMPROVEMENT:
                   best_val_reward = val_metrics['total_reward']
                   patience_counter = 0
               else:
                   patience_counter += 1
               
               if patience_counter >= PATIENCE:
                   print(f"Early stopping at episode {episode}")
                   break
       
       print("Training done")
       return training_history
   
   def _print_trade_list(self, test_env):
    print(f"\n=== TRADE LIST ===")
    if not hasattr(test_env, 'trade_history') or not test_env.trade_history:
        print("No trades exec")
        return
    
    for i, trade in enumerate(test_env.trade_history, 1):
        pnl = trade['pnl']
        # Get contracts from position_size (use absolute value)
        contracts = abs(trade.get('position_size', 0))
        print(f"Trade {i}: {contracts} contracts | P&L {pnl:+.2f}")
    
    total_pnl = sum(trade['pnl'] for trade in test_env.trade_history)
    print(f"Total trades: {len(test_env.trade_history)}")
    print(f"Total P&L: {total_pnl:+.2f}")
   
   def backtest(self, test_env) -> Dict:
       print("Starting backtesting...")
       
       self.ddqn.online_net.eval()
       
       results = self.evaluate_episode(test_env)
       
       performance = results['performance']
       actions = results['actions']
       
       action_counts = {}
       for action in actions:
           action_counts[action] = action_counts.get(action, 0) + 1
       
       backtest_results = {
           'performance_metrics': performance,
           'total_reward': results['total_reward'],
           'total_steps': results['steps'],
           'action_distribution': action_counts,
           'action_sequence': actions
       }
       
       print("\n" + "="*60)
       print("BACKTESTING RESULTS")
       print("="*60)
       
       print(f"Total Return: {performance.get('total_return', 0):.2%}")
       print(f"Sharpe Ratio: {performance.get('sharpe_ratio', 0):.2f}")
       print(f"Max Drawdown: {performance.get('max_drawdown', 0):.2%}")
       print(f"Win Rate: {performance.get('win_rate', 0):.1%}")
       print(f"Profit Factor: {performance.get('profit_factor', 0):.2f}")
       print(f"Total Trades: {performance.get('total_trades', 0)}")
       print(f"Final Equity: ${performance.get('final_equity', 0):,.2f}")
       
       self._print_trade_list(test_env)

       test_env.debug_pnl_calculation()



       print(f"\nAction Distribution:")
       for action, count in action_counts.items():
           action_pct = POSITION_ACTIONS[action]
           print(f"  Action {action} ({action_pct:+.0%}): {count} times ({count/len(actions):.1%})")
       
       return backtest_results
   
   def plot_training_curves(self, training_history: Dict, save_path: str = None):
       fig, axes = plt.subplots(2, 2, figsize=(15, 10))
       
       axes[0, 0].plot(training_history['train_rewards'], label='Train', alpha=0.7)
       axes[0, 0].plot(training_history['val_rewards'], label='Validation', alpha=0.7)
       axes[0, 0].set_title('Episode Rewards')
       axes[0, 0].set_xlabel('Episode')
       axes[0, 0].set_ylabel('Total Reward')
       axes[0, 0].legend()
       axes[0, 0].grid(True)
       
       axes[0, 1].plot(training_history['losses'], color='red', alpha=0.7)
       axes[0, 1].set_title('Training Loss')
       axes[0, 1].set_xlabel('Episode')
       axes[0, 1].set_ylabel('Average Loss')
       axes[0, 1].grid(True)
       
       axes[1, 0].plot(training_history['epsilons'], color='green', alpha=0.7)
       axes[1, 0].set_title('Epsilon Decay')
       axes[1, 0].set_xlabel('Episode')
       axes[1, 0].set_ylabel('Epsilon')
       axes[1, 0].grid(True)
       
       if training_history['val_performance']:
           returns = [p.get('total_return', 0) for p in training_history['val_performance']]
           sharpes = [p.get('sharpe_ratio', 0) for p in training_history['val_performance']]
           
           ax2 = axes[1, 1]
           ax3 = ax2.twinx()
           
           line1 = ax2.plot(returns, 'b-', label='Total Return', alpha=0.7)
           line2 = ax3.plot(sharpes, 'r-', label='Sharpe Ratio', alpha=0.7)
           
           ax2.set_xlabel('Episode')
           ax2.set_ylabel('Total Return', color='b')
           ax3.set_ylabel('Sharpe Ratio', color='r')
           ax2.set_title('Validation Performance')
           
           lines = line1 + line2
           labels = [l.get_label() for l in lines]
           ax2.legend(lines, labels, loc='upper left')
           ax2.grid(True)
       
       plt.tight_layout()
       
       if save_path:
           plt.savefig(save_path, dpi=300, bbox_inches='tight')
       
       if PLOT_TRAINING_CURVES:
           plt.show()

def create_agent_from_env(env) -> DDQNTradingAgent:
   state = env.reset()
   state_size = len(state)
   
   market_size = env.data_sequences.shape[1] * env.data_sequences.shape[2]
   trading_state_size = state_size - market_size
   
   agent = DDQNTradingAgent(
       market_feature_dim=env.data_sequences.shape[1],
       trading_state_dim=trading_state_size,
       action_space_size=env.get_action_space_size(),
       lookback_window=env.data_sequences.shape[2]
   )
   
   return agent

In [ ]:
# =============================================================================
# MAIN EXECUTION FUNCTIONS
# =============================================================================

def create_directories():
   directories = [MODEL_SAVE_PATH, LOG_SAVE_PATH, RESULTS_SAVE_PATH]
   for directory in directories:
       os.makedirs(directory, exist_ok=True)
   print("directories created")

def load_and_prepare_data(csv_path: str):
      
   data_handler = TradingDataHandler(csv_path)
   
   raw_data = data_handler.load_data()
   processed_data = data_handler.preprocess_data()
   
   sequences, targets, dates = data_handler.create_sequences(
       processed_data, 
       lookback=LOOKBACK_WINDOW
   )
   
   data_splits = data_handler.split_data(
       sequences, targets, dates, 
       quick_test=QUICK_TEST_MODE
   )
   
   feature_info = data_handler.get_feature_info()
   print(f"\nData Information:")
   print(f"  Features: {feature_info['n_features']}")
   print(f"  Samples: {feature_info['n_samples']}")
   print(f"  Date range: {feature_info['date_range'][0]} to {feature_info['date_range'][1]}")
   print(f"  Lookback window: {LOOKBACK_WINDOW} days")
   
   return data_splits, data_handler

def create_environments(data_splits):
   
   env_wrapper = TradingEnvironmentWrapper(data_splits)
   
   train_env = env_wrapper.create_env('train')
   val_env = env_wrapper.create_env('val')
   test_env = env_wrapper.create_env('test')
   
   env_info = env_wrapper.get_env_info()
   for split, info in env_info.items():
       print(f"\n{split.upper()} Environment:")
       print(f"  Samples: {info['n_samples']}")
       print(f"  State size: {info['state_size']}")
       print(f"  Action space: {info['action_space_size']}")
       print(f"  Date range: {info['date_range'][0]} to {info['date_range'][1]}")
   
   return env_wrapper, train_env, val_env, test_env

def train_agent(train_env, val_env):
   
   agent = create_agent_from_env(train_env)
   
   num_episodes = QUICK_TEST_EPISODES if QUICK_TEST_MODE else EPISODES
   
   print(f"training for {num_episodes} episodes...")
   
   training_history = agent.train(train_env, val_env, num_episodes)
   
   if PLOT_TRAINING_CURVES:
       save_path = f"{RESULTS_SAVE_PATH}training_curves.png" if SAVE_PLOTS else None
       agent.plot_training_curves(training_history, save_path)
   
   return agent, training_history

def backtest_agent(agent, test_env):
   
   backtest_results = agent.backtest(test_env)
   
   return backtest_results

def plot_backtest_results(test_env, backtest_results):
   if not PLOT_TRADING_RESULTS:
       return
   
   fig, axes = plt.subplots(2, 2, figsize=(16, 12))
   
   equity_curve = test_env.equity_curve
   dates = test_env.data_dates[:len(equity_curve)]
   
   axes[0, 0].plot(dates, equity_curve, linewidth=2, alpha=0.8)
   axes[0, 0].set_title('Equity Curve', fontsize=14, fontweight='bold')
   axes[0, 0].set_xlabel('Date')
   axes[0, 0].set_ylabel('Equity ($)')
   axes[0, 0].grid(True, alpha=0.3)
   axes[0, 0].tick_params(axis='x', rotation=45)
   
   peak = np.maximum.accumulate(equity_curve)
   drawdown = (peak - equity_curve) / peak
   
   axes[0, 1].fill_between(dates, drawdown, 0, alpha=0.7, color='red')
   axes[0, 1].set_title('Drawdown', fontsize=14, fontweight='bold')
   axes[0, 1].set_xlabel('Date')
   axes[0, 1].set_ylabel('Drawdown (%)')
   axes[0, 1].yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: '{:.1%}'.format(y)))
   axes[0, 1].grid(True, alpha=0.3)
   axes[0, 1].tick_params(axis='x', rotation=45)
   
   action_dist = backtest_results['action_distribution']
   actions = list(action_dist.keys())
   counts = list(action_dist.values())
   action_labels = [f"Action {a}\n({POSITION_ACTIONS[a]:+.0%})" for a in actions]
   
   axes[1, 0].bar(action_labels, counts, alpha=0.8)
   axes[1, 0].set_title('Action Distribution', fontsize=14, fontweight='bold')
   axes[1, 0].set_xlabel('Action')
   axes[1, 0].set_ylabel('Frequency')
   axes[1, 0].tick_params(axis='x', rotation=45)
   axes[1, 0].grid(True, alpha=0.3)
   
   performance = backtest_results['performance_metrics']
   metrics_names = ['Total Return', 'Sharpe Ratio', 'Max Drawdown', 'Win Rate', 'Profit Factor']
   metrics_values = [
       performance.get('total_return', 0),
       performance.get('sharpe_ratio', 0),
       performance.get('max_drawdown', 0),
       performance.get('win_rate', 0),
       performance.get('profit_factor', 0)
   ]
   
   formatted_values = [
       f"{metrics_values[0]:.1%}",
       f"{metrics_values[1]:.2f}",
       f"{metrics_values[2]:.1%}",
       f"{metrics_values[3]:.1%}",
       f"{metrics_values[4]:.2f}"
   ]
   
   bars = axes[1, 1].bar(metrics_names, metrics_values, alpha=0.8)
   axes[1, 1].set_title('Performance Metrics', fontsize=14, fontweight='bold')
   axes[1, 1].set_ylabel('Value')
   axes[1, 1].tick_params(axis='x', rotation=45)
   axes[1, 1].grid(True, alpha=0.3)
   
   for bar, formatted_val in zip(bars, formatted_values):
       height = bar.get_height()
       axes[1, 1].text(bar.get_x() + bar.get_width()/2., height + max(metrics_values)*0.01,
                      formatted_val, ha='center', va='bottom', fontweight='bold')
   
   plt.tight_layout()
   
   if SAVE_PLOTS:
       plt.savefig(f"{RESULTS_SAVE_PATH}backtest_analysis.png", 
                  dpi=300, bbox_inches='tight')
   
   plt.show()

def main(csv_path: str):
    print("="*60)
    print("DDQN TRADING SYSTEM - GOLDMAN ROLL STRATEGY")
    print("="*60)
    
    create_directories()
    
    np.random.seed(42)
    torch.manual_seed(42)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(42)
    
    try:
        data_splits, data_handler = load_and_prepare_data(csv_path)
    except Exception as e:
        print(f"Error loading data: {e}")
        return None, None, None  # Return None values instead of just return
    
    env_wrapper, train_env, val_env, test_env = create_environments(data_splits)
    
    try:
        agent, training_history = train_agent(train_env, val_env)
    except Exception as e:
        print(f"Error during training: {e}")
        return None, None, None  # Return None values instead of just return
    
    try:
        backtest_results = backtest_agent(agent, test_env)
    except Exception as e:
        print(f"Error during backtesting: {e}")
        return None, None, None  # Return None values instead of just return
    
    try:
        plot_backtest_results(test_env, backtest_results)
    except Exception as e:
        print(f"Error plotting results: {e}")
    
    print("\n" + "="*60)
    print("COMPLETED")
    print("="*60)
    
    return agent, backtest_results, training_history

In [8]:
# =============================================================================
# MULTI-RUN EXECUTION WITH DIFFERENT SEEDS
# =============================================================================

def run_with_seed(csv_path: str, seed: int, run_number: int):

    print(f"\n{'='*80}")
    print(f"START RUN {run_number}/5 WITH SEED {seed}")
    print(f"{'='*80}")
    
    # Set seeds
    np.random.seed(seed)
    torch.manual_seed(seed)
    random.seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)
    
    # Run the main function
    result = main(csv_path)
    
    if result[0] is not None:
        agent, backtest_results, training_history = result
        performance = backtest_results['performance_metrics']
        
        # Extract key metrics
        run_results = {
            'seed': seed,
            'run': run_number,
            'total_return': performance.get('total_return', 0),
            'sharpe_ratio': performance.get('sharpe_ratio', 0),
            'max_drawdown': performance.get('max_drawdown', 0),
            'win_rate': performance.get('win_rate', 0),
            'profit_factor': performance.get('profit_factor', 0),
            'total_trades': performance.get('total_trades', 0),
            'final_equity': performance.get('final_equity', 0)
        }
        
        print(f"\nRUN {run_number} COMPLETED SUCCESSFULLY!")
        print(f"Total Return: {run_results['total_return']:.2%}")
        print(f"Sharpe Ratio: {run_results['sharpe_ratio']:.2f}")
        print(f"Max Drawdown: {run_results['max_drawdown']:.2%}")
        
        return run_results
    else:
        print(f"RUN {run_number} FAILED!")
        return None

def run_multiple_seeds(csv_path: str):
    #5 experiments with different seeds and summary
    seeds = [42, 123, 456, 789, 999]
    all_results = []
    successful_runs = 0
    
    print("5 runs start")
    
    # Run 5 experiments
    for i, seed in enumerate(seeds, 1):
        result = run_with_seed(csv_path, seed, i)
        if result is not None:
            all_results.append(result)
            successful_runs += 1
    
    # Calculate summary statistics
    if successful_runs > 0:
        print(f"\n{'='*80}")
        print(f"SUMMARY - {successful_runs}/5 SUCCESSFUL RUNS")
        print(f"{'='*80}")
        
        # Calculate averages and std deviations
        metrics = ['total_return', 'sharpe_ratio', 'max_drawdown', 'win_rate', 
                  'profit_factor', 'total_trades', 'final_equity']
        
        summary = {}
        for metric in metrics:
            values = [r[metric] for r in all_results if r[metric] is not None]
            if values:
                summary[f'{metric}_mean'] = np.mean(values)
                summary[f'{metric}_std'] = np.std(values)
                summary[f'{metric}_min'] = np.min(values)
                summary[f'{metric}_max'] = np.max(values)
        
        # Print detailed results
        print("\nINDIVIDUAL RUN RESULTS:")
        print("-" * 100)
        print(f"{'Run':<5} {'Seed':<6} {'Return':<10} {'Sharpe':<8} {'Drawdown':<12} {'Win Rate':<10} {'Trades':<8}")
        print("-" * 100)
        
        for result in all_results:
            print(f"{result['run']:<5} {result['seed']:<6} "
                  f"{result['total_return']:<10.2%} {result['sharpe_ratio']:<8.2f} "
                  f"{result['max_drawdown']:<12.2%} {result['win_rate']:<10.1%} "
                  f"{result['total_trades']:<8.0f}")
        
        print("\nSUMMARY STATISTICS:")
        print("-" * 60)
        print(f"Average Return: {summary['total_return_mean']:.2%} ± {summary['total_return_std']:.2%}")
        print(f"Average Sharpe: {summary['sharpe_ratio_mean']:.2f} ± {summary['sharpe_ratio_std']:.2f}")
        print(f"Average Drawdown: {summary['max_drawdown_mean']:.2%} ± {summary['max_drawdown_std']:.2%}")
        print(f"Average Win Rate: {summary['win_rate_mean']:.1%} ± {summary['win_rate_std']:.1%}")
        print(f"Average Trades: {summary['total_trades_mean']:.0f} ± {summary['total_trades_std']:.0f}")
        
        print(f"\nRANGES:")
        print(f"Return Range: {summary['total_return_min']:.2%} to {summary['total_return_max']:.2%}")
        print(f"Sharpe Range: {summary['sharpe_ratio_min']:.2f} to {summary['sharpe_ratio_max']:.2f}")
        
        return all_results, summary
    else:
        print("ALL RUNS FAILED!")
        return [], {}


def run_multiple_csvs_and_seeds(csv_files: list):
    #5 experiments with different seeds for multiple CSV files
    
    all_results_by_file = {}
    all_summaries_by_file = {}
    
    print(f"STARTING MULTI-CSV STATISTICAL VALIDATION")
    print(f"Files to process: {len(csv_files)}")
    print(f"Runs per file: 5")
    print(f"Total experiments: {len(csv_files) * 5}")
    
    for file_idx, csv_file in enumerate(csv_files, 1):
        print(f"\n{'='*100}")
        print(f"PROCESSING FILE {file_idx}/{len(csv_files)}: {csv_file}")
        print(f"{'='*100}")
        
        if not os.path.exists(csv_file):
            print(f"ERROR: CSV file not found at: {csv_file}")
            continue
        
        # Run 5 seeds for this CSV file
        results, summary = run_multiple_seeds(csv_file)
        
        # Store results
        file_name = os.path.basename(csv_file).replace('.csv', '')
        all_results_by_file[file_name] = results
        all_summaries_by_file[file_name] = summary
        
        # Print file summary
        if results:
            print(f"\n{'-'*60}")
            print(f"SUMMARY FOR {file_name}")
            print(f"{'-'*60}")
            avg_return = summary.get('total_return_mean', 0)
            avg_sharpe = summary.get('sharpe_ratio_mean', 0)
            successful_runs = len(results)
            print(f"Successful runs: {successful_runs}/5")
            print(f"Average return: {avg_return:.2%}")
            print(f"Average Sharpe: {avg_sharpe:.2f}")
        else:
            print(f"NO SUCCESSFUL RUNS FOR {file_name}")
    
    # Print overall summary
    print_overall_summary(all_results_by_file, all_summaries_by_file)
    
    return all_results_by_file, all_summaries_by_file

def print_overall_summary(all_results_by_file: dict, all_summaries_by_file: dict):
    
    print(f"\n{'='*100}")
    print(f"OVERALL SUMMARY ACROSS ALL FILES")
    print(f"{'='*100}")
    
    # Summary table
    print(f"\nFILE PERFORMANCE SUMMARY:")
    print("-" * 120)
    print(f"{'File':<20} {'Runs':<6} {'Avg Return':<12} {'Avg Sharpe':<12} {'Avg Drawdown':<14} {'Avg Trades':<12}")
    print("-" * 120)
    
    all_returns = []
    all_sharpes = []
    total_successful_runs = 0
    
    for file_name, summary in all_summaries_by_file.items():
        if summary:  # If file had successful runs
            runs = len(all_results_by_file[file_name])
            avg_return = summary.get('total_return_mean', 0)
            avg_sharpe = summary.get('sharpe_ratio_mean', 0)
            avg_drawdown = summary.get('max_drawdown_mean', 0)
            avg_trades = summary.get('total_trades_mean', 0)
            
            print(f"{file_name:<20} {runs:<6} {avg_return:<12.2%} {avg_sharpe:<12.2f} "
                  f"{avg_drawdown:<14.2%} {avg_trades:<12.0f}")
            
            all_returns.append(avg_return)
            all_sharpes.append(avg_sharpe)
            total_successful_runs += runs
        else:
            print(f"{file_name:<20} {'0':<6} {'N/A':<12} {'N/A':<12} {'N/A':<14} {'N/A':<12}")
    
    print("-" * 120)
    
    # Overall statistics
    if all_returns:
        print(f"\nOVERALL STATISTICS:")
        print(f"Total successful runs: {total_successful_runs}")
        print(f"Files with successful runs: {len([s for s in all_summaries_by_file.values() if s])}")
        print(f"Average return across all files: {np.mean(all_returns):.2%}")
        print(f"Average Sharpe across all files: {np.mean(all_sharpes):.2f}")
        print(f"Best performing file (return): {max(all_returns):.2%}")
        print(f"Worst performing file (return): {min(all_returns):.2%}")
    else:
        print(f"\nNO SUCCESSFUL RUNS ACROSS ANY FILES!")

# =============================================================================
if __name__ == "__main__":
    import random
    
    csv_files = [
        "/notebooks/CL12 - Sheet1.csv",    # Brent Oil
        "/notebooks/CO12 - Sheet1.csv",    # Crude Oil
        "/notebooks/GC12 - Sheet1.csv",    # Gold
        "/notebooks/LC12 - Sheet1.csv",    # Live Cattle
        "/notebooks/SB12 - Sheet1.csv",    # Sugar #11
        "/notebooks/SI12 - Sheet1.csv",    # Silver
    ]
    
    # Check which files exist
    existing_files = []
    for file_path in csv_files:
        if os.path.exists(file_path):
            existing_files.append(file_path)
            print(f"✓ Found: {file_path}")
        else:
            print(f"✗ Missing: {file_path}")
    
    if existing_files:
        print(f"\nProcessing {len(existing_files)} files...")
        all_results, all_summaries = run_multiple_csvs_and_seeds(existing_files)
    else:
        print("ERROR: No CSV files found!")

